In [ ]:
import os
os.chdir('..')  # Muda para o diretório raiz do projeto
from main import funcoes as fc
import pyautogui



In [ ]:
import numpy as np
import os
import pandas as pd
import pyautogui
import pyperclip
import time
import tkinter as tk
import undetected_chromedriver as uc
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tabulate import tabulate
from tkinter import scrolledtext
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
#INSTALA O CHROME DRIVEr MAIS ATUALIZADO
# Desativa o bloqueio de pop-ups
options = webdriver.ChromeOptions()
options.add_argument("--disable-popup-blocking") 
#DEFINE O TEMPO DE EXECUÇÃO PARA CADA COMANDO DO PYAUTOGUI
pyautogui.PAUSE = 0.7
#INICIA O NAVEGADOR
# Caminho do ChromeDriver local
# chrome_driver_path = r"main\chromedriver-win64\chromedriver.exe"  # Altere para o caminho correto do seu ChromeDriver
# Configura o serviço do ChromeDriver
servico = Service(ChromeDriverManager().install())
# Inicia o navegador Chrome
navegador = webdriver.Chrome(service=servico, options=options)
navegador.maximize_window()
#ENTRA NO SEI
navegador.get('https://sei.anatel.gov.br/')


In [6]:
# navegador.switch_to.frame('ifrConteudoVisualizacao')
# navegador.switch_to.frame('ifrVisualizacao')

navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/div/a').click()

time.sleep(0.5)
                                #CLICA NA TAG DE INTERCORRENTE
navegador.find_element(By.PARTIAL_LINK_TEXT, 'Processo SEI ORCN - Pendência').click()

In [5]:
processo = '53500.088805/2024-88'
#RETORNA PARA A PAGINA INICIAL DO PROCESSO
navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processo)
elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
elementos.send_keys(Keys.ENTER)
time.sleep(1)

navegador.switch_to.default_content()
navegador.switch_to.frame('ifrArvore')
#CLICA NA DECLARACAO DE CONFORMIDADE
navegador.find_element(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade").click()
time.sleep(1)
navegador.switch_to.default_content()
navegador.switch_to.frame('ifrConteudoVisualizacao')
navegador.switch_to.frame('ifrVisualizacao')
#ARMAZENA O EMAIL DO SOLICITANTE
emailSol = navegador.find_element(By.XPATH, '/html/body/div[3]/a[1]').text
navegador.switch_to.default_content()
#RETORNA PARA A PAGINA INICIAL DO PROCESSO
navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processo)
elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
elementos.send_keys(Keys.ENTER)
time.sleep(1)
navegador.switch_to.frame('ifrConteudoVisualizacao')
#CLICA NO ICONE DE ENVIO DE EMAIL
fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/email_enviar.svg?18')]")
#clica_noelemento(navegador, By.XPATH, '//*[@id="divArvoreAcoes"]/a[10]')
time.sleep(1)
#VAI PARA A JANELA MAIS RECENTE ABERTA
navegador.switch_to.window(navegador.window_handles[-1])
#time.sleep(1)
#ABRE O DROPDOWN COM AS OPCOES DE EMAIL DA ANATEL
select_element = WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="selDe"]')))
#select_element = navegador.find_element(By.XPATH, '//*[@id="selDe"]')
select = Select(select_element)
#SELECIONA O EMAIL DA ANATEL
select.select_by_visible_text('ANATEL/E-mail de replicação <nao-responda@anatel.gov.br>')
#ESCREVE O EMAIL DO SOLICITANTE
navegador.find_element(By.XPATH, '//*[@id="s2id_autogen1"]').send_keys(emailSol)
time.sleep(1.2)
#CLICA NO EMAIL DO SOLICITANTE
fc.clica_noelemento(navegador, By.CLASS_NAME, 'select2-result-label')
#MOSTRA AS OPCOES DE EXIGENCIA
#EXECUTA A CONDICAO PARA CADA EXIGENCIA

In [6]:
#FUNCAO PARA ERROS NA DECLARACAO DE CONFORMIDADE PEDINDO PARA ABRIR NOVO PROCESSO SEI
def erro_declaracao(processos, impProp, navegador, jaHomologado=False):
    #ENVIA TEXTO DE INPUT PARA A VARIAVEL
    def get_text():
        nonlocal exig2
        exig2 = text_area.get("1.0", tk.END).strip()
        root.destroy()

    #INICIALIZA A VARIAVEL LOCAL QUE RECEBERA O TEXTO DE INPUT
    exig2 = ""

    #CRIA A JANELA DA INTERFACE PARA CAIXA DE INPUT
    root = tk.Tk()
    root.title("Inserir Texto de Exigência")

    #INSTRUCAO DO QUE O USUARIO DEVE FAZER
    label_instruction = tk.Label(root, text="Insira apenas a exigência:")
    label_instruction.pack(padx=10, pady=5)

    #CRIA AREA DE TEXTO COM ROLAGEM
    text_area = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=80, height=20)
    text_area.pack(padx=10, pady=10)

    #CRIA BOTAO DE RECEBER O TEXTO
    btn_get_text = tk.Button(root, text="Obter Texto", command=get_text)
    btn_get_text.pack(pady=5)

    #INICIA O LOOP PRINCIPAL QUE MANTEM A JANELA ATIVA
    root.mainloop()
    #VERIFICA SE É IMPORTADO PRA USO PRÓPRIO
    #CASO SEJA IMPORTADO PRA USO PROPRIO E SEJA UM RADIO ELE PASSARÁ O MANUAL PARA RADIO E CORRELATOS
    #SE FOR DRONE PRA USO PROPRIO ELE PASSARA O MANUAL PARA DRONE
    
    if impProp == 'Sim':
        while True:
            try:
                radioamador = str(input('O produto é um rádio ou correlatos? Se sim digite [1], se for um Drone para uso próprio digite [2]: '))
                
                if radioamador == '1':
                    link = 'https://docs.google.com/document/d/1Y-8wGVnDMuku_Dmd5uT6hHCuj3ruXbuW3o-JFDWxog0/edit'
                    break
                elif radioamador == '2':
                    link = 'https://docs.google.com/document/d/1YsrMwMxyVysCJ9VhtGOOU1S1FlbmkRFaYSjojLxXVsQ/edit'
                    break
                else:
                    print("Opção inválida. Por favor, digite [1] para rádio ou correlatos, ou [2] para Drone para uso próprio.")
            except ValueError:
                print("Entrada inválida. Por favor, digite um número [1] para rádio ou correlatos, ou [2] para Drone para uso próprio.")
    
    #CASO SEJA UMA DECLARACAO APENAS PARA DRONE ELE IRÁ PASSAR DIRETAMENTE O MANUAL PARA DRONE
    else:
        link = 'https://docs.google.com/document/d/1YsrMwMxyVysCJ9VhtGOOU1S1FlbmkRFaYSjojLxXVsQ/edit'
    
    #TESTA QUAL TIPO DE EXIGENCIA É
    if jaHomologado:
        texto_padrao = f'''Prezado(a) Senhor(a),

        O requerimento realizado nesta Declaração de Conformidade possui um produto que já possui o selo de homologação da Anatel, sendo esse produto:

        {exig2}

        Como o produto deve ser homologado apenas uma vez, solicitamos que realize um novo processo SEI contendo apenas os produtos não homologados previamente.

        Favor seguir as seguintes orientações:
        1. Realizar um novo processo SEI, cumprindo com a(s) exigência(s) apontada(s) anteriormente. Seguindo o manual: {link}

        2. Para criar um novo processo SEI, solicitamos acessar o sistema Sei da Anatel: https://sei.anatel.gov.br/sei/controlador_externo.php?acao=usuario_externo_logar&id_orgao_acesso_externo=0

        Dessa maneira, devido a presença de exigência(s), informamos que o atual processo SEI nº {processos} será arquivado.

        FAVOR NÃO RESPONDER ESTE E-MAIL.



        Atenciosamente,

        ORCN - Gerência de Certificação e Numeração

        SOR - Superintendência de Outorga e Recursos à Prestação

        Anatel - Agência Nacional de Telecomunicações'''

    else:
        texto_padrao = f'''Prezado(a) Senhor(a),

        Em atenção à demanda registrada no processo em referência, apresentamos as seguintes pendências observadas no documento de Declaração de Conformidade:

        {exig2}

        Favor seguir as seguintes orientações:
        
        1. Realizar um novo processo SEI, cumprindo com a(s) exigência(s) apontada(s) anteriormente. Seguindo o manual: {link}

        2. Para criar um novo processo SEI, solicitamos acessar o sistema Sei da Anatel: https://sei.anatel.gov.br/sei/controlador_externo.php?acao=usuario_externo_logar&id_orgao_acesso_externo=0

        Dessa maneira, devido a presença de exigência(s), informamos que o atual processo SEI nº {processos} será arquivado.

        FAVOR NÃO RESPONDER ESTE E-MAIL.



        Atenciosamente,

        ORCN - Gerência de Certificação e Numeração

        SOR - Superintendência de Outorga e Recursos à Prestação

        Anatel - Agência Nacional de Telecomunicações'''
        
    #INSERE ASSUNTO DO PROCESO
    navegador.find_element(By.ID, 'txtAssunto').send_keys(f"Processo SEI nº {processos} - Indeferido")
    #INSERE EMAIL COM CORPO DO EMAIL
    navegador.find_element(By.ID, 'txaMensagem').send_keys(texto_padrao)



In [7]:
erro_declaracao(processo, 'Sim', navegador, jaHomologado=True)